In [111]:
import gym.spaces

from imports import *

In [112]:
### Observations ###
etape_enchere = 0 ## est-ce qu'on est en train d'enchérir (0) de faire son chien (1)  ou de jouer (2)
etape_chien = 0
etape_jeu = 0
position_joueur = 0   ## ma position : 3 = donneur, ensuite de 0 à 2 en fonction du donneur
enchere = 0 ## 0 : passe, 1 : petite, 2 : garde, 3 : garde sans, 4 : garde contre
attacking_player = 0 ## le joueur qui a pris
joueur_attaque = 0 ## 1 : le joueur a remporté les enchères, 0 : le joueur est en défense


mes_trefles = []
mes_carreaux = []
mes_coeurs = []
mes_piques = []
mes_atouts = []
excuse = 0

tombes_trefles = []
tombes_carreaux = []
tombes_coeurs = []
tombes_piques = []
tombes_atouts = []
tombee_excuse = 0

chien_trefles = []
chien_carreaux = []
chien_coeurs = []
chien_piques = []
chien_atouts = []
chien_excuse = 0

points_attaquant, points_defense = 0, 0

spaces = {
            'observations': gym.spaces.MultiBinary(312),
            'metadonnees': gym.spaces.Box(low=np.array([0] * 9), high=np.array([91] * 6), dtype=np.int8)
        }

In [280]:
def hand_to_text(hand, player):
    equiv = {}
    for i in range(0, 10):
        equiv[i] = str(i+1)
    equiv[10] = "valet"
    equiv[11] = "cavalier"
    equiv[12] = "dame"
    equiv[13] = "roi"
    if player != "chien":
        print(f"Joueur {player} : ")
    else:
        print("Chien : ")
    print(f"Trèfles : {[equiv[i] for i in range(0, 14) if hand[i]]}")
    print(f"Carreaux : {[equiv[i] for i in range(0, 14) if hand[i+14]]}")
    print(f"Coeurs : {[equiv[i] for i in range(0, 14) if hand[i+28]]}")
    print(f"Piques : {[equiv[i] for i in range(0, 14) if hand[i+42]]}")
    print(f"Atouts : {[i for i in range(0, 22) if hand[i+56]]}")
    print('-----------------------------------------------------------------')

def generate_hands():
    all_cards = [1] * 78
    player1, player2, player3, player4, dog = [0] * 78, [0] * 78, [0] * 78, [0] * 78, [0] * 78
    for i in range(0, 18):
        to_give = random.choice([i for i in range(0, 78) if all_cards[i]])
        all_cards[to_give] = 0
        player1[to_give] = 1
        
        to_give = random.choice([i for i in range(0, 78) if all_cards[i]])
        all_cards[to_give] = 0
        player2[to_give] = 1
        
        to_give = random.choice([i for i in range(0, 78) if all_cards[i]])
        all_cards[to_give] = 0
        player3[to_give] = 1
        
        to_give = random.choice([i for i in range(0, 78) if all_cards[i]])
        all_cards[to_give] = 0
        player4[to_give] = 1
        
    for card_index in [i for i in range(0, 78) if all_cards[i]]:
        dog[card_index] = 1
        
    return [player1, player2, player3, player4, dog]

def generate_action_mask(main, couleur_demandee, atout_min):
    all_index = [i for i in range(0, 77) if main[i]]
    # Si aucune carte n'a encore été jouée
    if couleur_demandee == 0:
        cartes_jouables = [i for i in range(0, 77) if main[i]]
    # Si on est à une couleur
    elif couleur_demandee in [1, 2, 3, 4]:
        if [i for i in range(14 * (couleur_demandee - 1), 14 * couleur_demandee) if main[i]]:
            cartes_jouables = [i for i in range(14 * (couleur_demandee - 1), 14 * couleur_demandee) if main[i]]
        else:
            if [i for i in range(56, 77) if main[i]]:
                cartes_jouables = [i for i in range(56, 77) if main[i] and i-55 > atout_min]
                if not cartes_jouables:
                    cartes_jouables = [i for i in range(56, 77) if main[i]]
            else:
                cartes_jouables = [i for i in range(0, 77) if main[i]]
    # Si on est à l'atout
    else:
        cartes_jouables = [i for i in range(56, 77) if main[i] and i-55 > atout_min]
        if not cartes_jouables:
            cartes_jouables = [i for i in range(56, 77) if main[i]]
            if not cartes_jouables:
                cartes_jouables = [i for i in range(0, 77) if main[i]]
    if main[77] == 1:
                cartes_jouables.append(77)
        
    # Maintenant qu'on sait quelles cartes sont jouables, on transforme en action_mask
    action_mask = [0] * 24
    for i in range(len(all_index)):
        if all_index[i] in cartes_jouables:
            action_mask[i] = 1
    return action_mask

def enchere_model(model, mon_jeu, enchere_en_cours):
    if model == "random":
        return random.randint(enchere_en_cours, 4)
    
def constitution_chien(model, main, chien):
    main_globale = [main[i] + chien[i] for i in range(0, 78)]
    new_chien = [0] * 78
    if model == "random":
        for i in range(0, 6):
            to_chien = random.choice([i for i in range(0, 78) if main_globale[i]])
            new_chien[to_chien] = 1
            main_globale[to_chien] = 0
        return main_globale, new_chien
    
def jouer_carte(model, main, couleur_demandee, atout_min=0):
    cartes_jouables = [0] * 78
    carte_jouee = 0
    # Si aucune carte n'a encore été jouée
    if couleur_demandee == 0:
        cartes_jouables = [i for i in range(0, 77) if main[i]]
    # Si on est à une couleur
    elif couleur_demandee in [1, 2, 3, 4]:
        if [i for i in range(14 * (couleur_demandee - 1), 14 * couleur_demandee) if main[i]]:
            cartes_jouables = [i for i in range(14 * (couleur_demandee - 1), 14 * couleur_demandee) if main[i]]
        else:
            if [i for i in range(56, 77) if main[i]]:
                cartes_jouables = [i for i in range(56, 77) if main[i] and i-55 > atout_min]
                if not cartes_jouables:
                    cartes_jouables = [i for i in range(56, 77) if main[i]]
            else:
                cartes_jouables = [i for i in range(0, 77) if main[i]]
    # Si on est à l'atout
    else:
        cartes_jouables = [i for i in range(56, 77) if main[i] and i-55 > atout_min]
        if not cartes_jouables:
            cartes_jouables = [i for i in range(56, 77) if main[i]]
            if not cartes_jouables:
                cartes_jouables = [i for i in range(0, 77) if main[i]]
    if main[77] == 1:
                cartes_jouables.append(77)
    
    if model == "random":
        carte_jouee = random.choice(cartes_jouables)
        
    if couleur_demandee == 0 and carte_jouee != 77:
        if carte_jouee < 56:
            couleur_demandee = (carte_jouee // 14) + 1
        else:
            couleur_demandee = 5
    
    if 56 <= carte_jouee <= 77 and atout_min < carte_jouee - 56:
        atout_min = carte_jouee - 56
        
    return carte_jouee, couleur_demandee, atout_min
    
def mask_fn(env: gym.Env) -> np.ndarray:
    return env.valid_action_mask()

In [296]:
class TarotEnv(gym.Env):
    def __init__(self, best_model):

        # Observation space
        self._obs_space_in_preferred_format = True

        spaces = {
            'observations': gym.spaces.MultiBinary(312),
            'metadonnees': gym.spaces.Box(low=np.array([0] * 9), high=np.array([91] * 9), dtype=np.int8)
        }

        self.observation_space = gym.spaces.Dict(spaces)

        # Action space
        self._action_space_in_preferred_format = True
        self.action_space = gym.spaces.Discrete(24)
        # Initial state
        self.action_mask = np.array([1] + [0] * 23, dtype=np.int8)
        
        self.distribution = generate_hands()
        self.global_turn = 0
        
        
        self.pos_joueur, self.players_turn, self.etape, self.enchere, self.attacking_player = random.randint(0, 3), 0, 0, 0, 0
        self.couleur_demandee, self.atout_min = 0, 0
        self._state = {}
        # Set ending condition
        self.best_model = best_model

        super().__init__()

    def step(self, action):     
        
        ## Tour d'enchères
        if self._state["metadonnees"][0] == 1:
            if action != 0:
                self.enchere = action
                if self.enchere > self._state["metadonnees"][4]:
                    self.attacking_player = self.players_turn
                self._state["metadonnees"][4] = self.enchere
            self.players_turn += 1
            while self.players_turn < 4:
                self.enchere = enchere_model("random", self.distribution[self.players_turn], self.enchere)
                if self.enchere > self._state["metadonnees"][4]:
                    self.attacking_player = self.players_turn
                self._state["metadonnees"][4] = self.enchere
                self.players_turn += 1
            ## Si personne ne prend, on termine l'épisode et on en relance un autre (i.e. on redistribue)
            if self.enchere == 0:
                done = True
                reward = 0
            else:
                ## Si l'agent a la plus grosse enchère, il passe à la constitution du chien
                if self.attacking_player == self._state["metadonnees"][3]:
                    self.players_turn = 0
                    self._state["metadonnees"][0] = 0
                    self._state["metadonnees"][1] = 1
                    self._state["metadonnees"][6] = 1 # joueur_attaque = 1
                    
                    ## On interdit de mettre des rois ou des atouts au chien
                    main_globale = [self.distribution[self.attacking_player][i] + self.distribution[4][i] for i in range(0, 78)]
                    temp_mask = []
                    for i in range(0, 78):
                        if main_globale[i] == 1:
                            if i % 14 == 13:
                                temp_mask.append(0)
                            elif i >= 56:
                                if len([x for x in temp_mask if x == 0]) < 6 and i != 56 and i < 77:
                                    temp_mask.append(1)
                                else:
                                    temp_mask.append(0)
                            else:
                                temp_mask.append(1)
                    self.action_mask = np.array(temp_mask, dtype=np.int8)
                    
                    self.distribution[self.attacking_player] = main_globale
                    self.distribution[4] = [0] * 78
                    self._state["observations"][0:78] = main_globale
                    self._state["metadonnees"][6] = 1 # C'est notre agent qui attaque
                ## Sinon c'est les modèles adverses qui s'en occupent
                else:
                    new_distribution = constitution_chien("random", self.distribution[self.attacking_player], self.distribution[4])
                    self.distribution[self.attacking_player] = new_distribution[0]
                    self.distribution[4] = new_distribution[1]
                    self._state["metadonnees"][0] = 0
                    self._state["metadonnees"][2] = 1
                    self._state["metadonnees"][6] = 0 # joueur_attaque = 0 (i.e. l'agent est en defense)
                    
                    ## On commence le jeu de la carte
                    self.players_turn = 0
                    while self.players_turn < self._state["metadonnees"][3]:
                        carte_jouee, self.couleur_demandee, self.atout_min = jouer_carte("random", self.distribution[self.players_turn], self.couleur_demandee)
                        # On met à jour le plateau avec la carte jouee
                        self._state["observations"][carte_jouee + 234] = 1
                        self.distribution[self.players_turn][carte_jouee] = 0
                        self.players_turn += 1
                    self.action_mask = generate_action_mask(self._state["observations"][0:78], self.couleur_demandee, self.atout_min)
                        
                self._state["metadonnees"][5] = self.attacking_player # On indique quel joueur est à l'attaque
                done = False
        ## Constitution du chien
        elif self._state["metadonnees"][1] == 1:
            to_transfer = [i for i in range(0, 78) if self._state["observations"][i]][action]
            self._state["observations"][to_transfer] = 0
            self._state["observations"][to_transfer + 156] = 1
            self.action_mask = np.array([1] * len([i for i in range(0, 78) if self._state["observations"][i]]) + [0] * (24 - len([i for i in range(0, 78) if self._state["observations"][i]])))
            
            ## Si le chien est terminé, on lance la partie !
            if len([i for i in range(0, 78) if self._state["observations"][i + 156]]) == 6:
                self._state["metadonnees"][1] = 0
                self._state["metadonnees"][2] = 1
                self.players_turn = 0
                while self.players_turn < self._state["metadonnees"][3]:
                    carte_jouee, self.couleur_demandee, self.atout_min = jouer_carte("random", self.distribution[self.players_turn], self.couleur_demandee)
                    # On met à jour le plateau avec la carte jouee
                    self._state["observations"][carte_jouee + 234] = 1
                    self.distribution[self.players_turn][carte_jouee] = 0
                    self.players_turn += 1
                self.action_mask = generate_action_mask(self._state["observations"][0:78], self.couleur_demandee, self.atout_min)
            done = False
        else:
            # Le jeu de la carte à proprement parler
            mes_cartes = [i for i in range(0, 78) if self._state["observations"][i]]
            carte_jouee = mes_cartes[action]
        
            if self.couleur_demandee == 0 and carte_jouee != 77:
                if carte_jouee < 56:
                    self.couleur_demandee = (carte_jouee // 14) + 1
                else:
                    self.couleur_demandee = 5
            
            if 56 <= carte_jouee <= 77 and self.atout_min < carte_jouee - 56:
                self.atout_min = carte_jouee - 56
                
            self._state["observations"][carte_jouee + 234] = 1
            self.distribution[self.players_turn][carte_jouee] = 0
            self.players_turn += 1
            
            while self.players_turn < 4:
                carte_jouee, self.couleur_demandee, self.atout_min = jouer_carte("random", self.distribution[self.players_turn], self.couleur_demandee)
                # On met à jour le plateau avec la carte jouee
                self._state["observations"][carte_jouee + 234] = 1
                self.distribution[self.players_turn][carte_jouee] = 0
                self.players_turn += 1
                
            self.players_turn = 0
            self.global_turn += 1
            
            if self.global_turn < 18:
                while self.players_turn < self._state["metadonnees"][3]:
                    carte_jouee, self.couleur_demandee, self.atout_min = jouer_carte("random", self.distribution[self.players_turn], self.couleur_demandee)
                    # On met à jour le plateau avec la carte jouee
                    self._state["observations"][carte_jouee + 234] = 1
                    self.distribution[self.players_turn][carte_jouee] = 0
                    self.players_turn += 1
                
            self.action_mask = generate_action_mask(self._state["observations"][0:78], self.couleur_demandee, self.atout_min)
            
            done = False
            
        if self.global_turn == 18:
            done = True


        self._state["metadonnees"][0] = self.etape
        truncated = done
        reward = 0

        # Return step information
        return (
            self._state,
            reward,
            done,
            truncated,
            {},
        )

    def reset(self, *, seed=None, options=None):
        ## On initialise les vecteurs d'état à 0
        self._state["observations"] = np.array([0] * 312, dtype=np.int8)
        self._state["metadonnees"] = np.array([0] * 9, dtype=np.int8)
        
        ## Distribution des différentes mains et tirage au sort de la position de l'agent
        self.distribution = generate_hands()
        self.pos_joueur = random.randint(0, 3)
        self.players_turn = 0
        self.etape, self.global_turn = 0, 0
        self.enchere = 0
        self.attacking_player = 0
        self.couleur_demandee, self.atout_min = 0, 0
        
        ## Attribution de la main au joueur
        self._state["observations"][0:78] = self.distribution[self.pos_joueur]
        
        ## Enchères des adversaires
        while self.players_turn < self.pos_joueur:
            self.enchere = enchere_model("random", self.distribution[self.players_turn], self.enchere)
            if self.enchere > self._state["metadonnees"][2]:
                self.attacking_player = self.players_turn
            self.players_turn += 1
            self._state["metadonnees"][4] = self.enchere
            
        # Mise à jour des métadonnées
        self._state["metadonnees"][0] = 1
        self._state["metadonnees"][3] = self.pos_joueur
            
        # Mise à jour de l'action mask
        self.action_mask = np.array([1] + [0] * self.enchere + [1] * (4 - self.enchere) + [0] * 19, dtype=np.int8)
        
        return self._state, {}

    def valid_action_mask(self):
        return self.action_mask

In [297]:
env = TarotEnv(best_model="random")

env = ActionMasker(env, mask_fn)

# It will check your custom environment and output additional warnings if needed
model = MaskablePPO(MaskableMultiInputActorCriticPolicy,
                        env,
                        verbose=1,
                        tensorboard_log="./first_test_skyjo",
                        learning_rate=0.0002,
                        n_epochs=15,
                        gamma=0.995,
                        clip_range=0.1,
                        batch_size=128)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [298]:
model.learn(50, reset_num_timesteps=True)

Logging to ./first_test_skyjo\PPO_55
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.8     |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 1272     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
